In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import dill
import numpy as np
import scipy.io as io

sys.path.append('../helper_functions')
sys.path.append('helper_functions')
sys.path.append('../objects')
sys.path.append('objects')

In [3]:
from FishGeneration import FishGeneration
from Fish_IceCreamCone import IceCreamConeFish
from Fish_Capsule import CapsuleFish

from assertion_tests import run_tests
run_tests()

'Success!'

## Load fish details

#### the EOD waveform

In [4]:
# use a dummy EOD until real EOD is available in this framework
full_eod = np.linspace(0, 2 * np.pi, 1000)
full_eod = np.sin(full_eod) * full_eod * np.exp(-((full_eod - np.pi) ** 2) / (2 * (np.pi / 6) ** 2))

#### the receptor filters

In [5]:
# use dummy filters until real filters are available in this framework
receptor_filters = np.random.randn(2, *full_eod.shape)
receptor_filters.shape

(2, 1000)

#### check the input arguments for the wanted fish object (helps with creating fish objects)

In [7]:
for x in CapsuleFish._initialize_input_argument_names():
    print(f'{x},')

conductivity=1,
relative_permittivity=1,
receptors_locations=np.array([]).reshape(0,3),
receptors_normals=np.array([]).reshape(0,3),
point_currents_magnitudes=np.array([]).reshape(0),
point_currents_locations=np.array([]).reshape(0,3),
receptor_filters=np.array([]).reshape(0,1000),
eod_wave_form=np.arange(1000),
skin_resistivity=1,
sampling_rate=(2.5, "M"),  # MHz,
eod_delay=(0,), # s,
nose_position=[0,0,0],
fish_length=1,
angle_yaw=0,
angle_pitch=0,
angle_roll=0,
relative_bend_locations_percentage=np.array([]),
relative_bend_angle_lateral=(np.array([]),"deg"),
relative_bend_angle_dorso_ventral =(np.array([]),"deg"),
point_currents_range_percentage=dict(start=0, end=100),
N_point_currents=101,
point_currents_magnitude_scale=1,
receptors_init=dict(method="random", head=100, body=200),
vertical_semi_axis,
lateral_semi_axis=None,
rostrocaudal_semi_axis=None,
rostrocaudal_semi_axis_tail=None,


## Dataset: YYYY-MM-DD Purpose

#### create the Fish object

In [19]:
base_fish = CapsuleFish(
    receptor_filters=receptor_filters,
    eod_wave_form=full_eod,
    skin_resistivity=(3, "k", "c.2"),  # 3 kOhm.cm^2
    sampling_rate=(2.5, "M"),  # 2.5 MHz,
    nose_position=[0,0,0],
    fish_length=(15, 'c'),
    angle_yaw=(0, 'deg'),
    angle_pitch=(0, 'deg'),
    angle_roll=(0, 'deg'),
    relative_bend_locations_percentage=np.array([]),
    relative_bend_angle_lateral=(np.array([]),"deg"),
    relative_bend_angle_dorso_ventral =(np.array([]),"deg"),
    receptors_init=dict(method='grid_uniDense',
                        head_t=0, head_u=0,
                        tail_t=0, tail_u=0,
                        body_t=60, body_u=30,),
    vertical_semi_axis=(2.0, 'c'),
    lateral_semi_axis=(0.9, 'c'),
    rostrocaudal_semi_axis=0,
    rostrocaudal_semi_axis_tail=0,
)      

# dill.dump(base_fish, open('../data/baseFish/data_240425_Base_Detection_1050_receptors.pkl', 'wb'), protocol=4)
# base_fish = dill.load(open('../data/baseFish/data_220623_Base_fish_with_Single_Receptor.pkl', 'rb'))

Success!


In [22]:
base_fish.visualize_scatter(show_normals=2, show_point_currents=3, fig_width=1000, fig_height=350)
print(base_fish.get_N_receptors())

1830


In [ ]:
# Create Data Params Dict
data_params_dict = {}
# data_params_dict['save_file_name']                  = 'data_240411_SpatialProcessing_MovingObject_7by15'
data_params_dict['save_file_name']                  = 'data_240425_Detection_1050_receptors_HighContrast'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
# data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_resistances']                = np.array([2e3, 100e3])  # HighContrast
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10  # HighContrast when paired with HighContrast resistances
# data_params_dict['worm_capacitances']               = np.array([2e-10])  # HighContrast
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3   # - np.arange(101) * 3e-4
data_params_dict['worm_position_ys']                = (5 * np.power(np.sqrt(1.33), np.arange(25))).round(0) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_231116_SpatialProcessing_MovingObject_11by25'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]  # np.array([100, 300, 600]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
                                                      # [[-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3),
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3), 
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3)]
                                                    #  [[-1e9]+list(-np.linspace(5*base_fish.roc_ax, base_fish.roc_ax, 9)), ]  # 
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9)),
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9))]
data_params_dict['boundary_normals']                =  [[0,0,1]]
                                                      # [[0,0,1], [0,-1,0], [0,1,0]]
      #  [[0,0,1], ]  #
      #    [0,-1,0],
      #    [0,1,0]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]#[[x] for x in np.linspace(-60,60,25)]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]#[[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]#[[60]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3 - np.arange(11) * 3e-3
data_params_dict['worm_position_ys']                = np.array([5,10,15,30,50,70,100,150]) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_240311_falloff_with_distance_and_radius'
# aquarium properties
data_params_dict['water_conductivities']            = np.array([100]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.array([500,50000])
data_params_dict['worm_capacitances']               = np.array([2e-10])
data_params_dict['worm_radii']                      = np.arange(5,11) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3
data_params_dict['worm_position_ys']                = np.arange(10,201) * 1e-3 + base_fish.lat_ax
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)